# 🎯 ES診断ツール - Google Colab起動用（SimCSE-JA版）

このノートブックを使用して、ES診断ツール（SimCSE-JAモデル版）をGoogle Colab上で起動します。

**使用モデル**: pkshatech/simcse-ja-bert-base-clcmlp（日本語特化・768次元）

## 📋 実行手順

1. **セル1**: パッケージのインストール
2. **セル2**: GitHubリポジトリをクローン
3. **セル3**: CSVデータの準備（Google Drive / アップロード）
4. **セル4**: 前処理済みデータの確認・読み込み（オプション・高速化）
5. **セル5**: アプリケーションの起動
6. **セル6**: 前処理済みデータの保存（オプション・次回起動を高速化）

## ⚡ 高速起動について

**初回起動**: セル1〜5を順番に実行（3-5分）
**2回目以降**: セル6で保存した前処理済みデータを使用して数秒で起動！

---

## セル1: 必要なパッケージをインストール

In [ ]:
# 必要なパッケージをインストール
!pip install fastapi uvicorn pandas numpy scikit-learn tqdm -q

# sentence-transformers をインストール（セマンティック検索用）
print("🔧 sentence-transformersをインストール中...")
!pip install sentence-transformers -q
print("✅ sentence-transformersのインストールが完了しました")

# pyngrokを個別にインストール（タイムアウト対策）
!pip install pyngrok -q --default-timeout=100

print("✅ 全パッケージのインストールが完了しました")
print("\n📦 インストールされたパッケージ:")
print("  - FastAPI + uvicorn (Webアプリケーション)")
print("  - pandas (データ処理)")
print("  - numpy (数値計算)")
print("  - scikit-learn (TF-IDF、機械学習)")
print("  - sentence-transformers (セマンティック検索用BERT)")
print("  - tqdm (進捗表示)")
print("  - pyngrok (公開URL作成)")
print("\n⚡ パフォーマンス最適化:")
print("  - SimCSE-JAモデル使用: 768次元の日本語特化エンベディング")
print("  - バッチ処理で高速化")
print("  - GPU対応（T4有効時はさらに高速）")

## セル2: GitHubリポジトリをクローン

In [ ]:
# リポジトリをクローン
!git clone https://github.com/ryosuke-fujii/es.git

# カレントディレクトリを変更
%cd es

print("✅ リポジトリのクローンが完了しました")
print("\n📁 ディレクトリ構造:")
!ls -la

## セル3: CSVデータの準備

以下の2つの方法から選択してください：
- **方法A**: Google Driveから読み込む（推奨）
- **方法B**: 直接アップロード

In [ ]:
# ============================================
# 方法A: Google Driveから読み込む（推奨）
# ============================================

from google.colab import drive
import os

# Google Driveをマウント
drive.mount('/content/drive')

# 統合CSVファイルのパスを指定（就活会議 + ワンキャリア）
csv_path = "/content/drive/MyDrive/企画・マーケチーム/gaxi_自動化/Python/ESデータ整形/unified_es_data.csv"

# ファイルが存在するか確認
if os.path.exists(csv_path):
    print(f"✅ ファイルを発見: {csv_path}")
    # データソース情報を確認
    import pandas as pd
    df = pd.read_csv(csv_path)
    print(f"📊 データ件数: {len(df)}件")
    if 'data_source' in df.columns:
        print("\nデータソース内訳:")
        print(df['data_source'].value_counts())
else:
    print(f"❌ ファイルが見つかりません: {csv_path}")
    print("\n💡 ヒント: ファイルブラウザで目的のファイルを右クリック → 'パスをコピー'")

In [ ]:
# ============================================
# 方法B: 直接アップロード
# ============================================

from google.colab import files

# ファイルをアップロード
uploaded = files.upload()

# アップロードされたファイル名を取得
csv_path = list(uploaded.keys())[0]
print(f"✅ ファイルをアップロードしました: {csv_path}")

## セル4: 前処理済みデータの確認・読み込み（オプション）

前回実行時に保存した前処理済みデータがあれば読み込み、なければ新規作成します。
これにより2回目以降の起動が高速化されます（3-5分 → 数秒）。

In [ ]:
import os
import sys

# srcディレクトリをPythonパスに追加
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# ============================================
# 前処理済みデータのパス設定
# ============================================
# Google Driveの指定されたディレクトリを使用
preprocessed_dir = "/content/drive/MyDrive/企画・マーケチーム/gaxi_自動化/Python/ESデータ整形/es_preprocessed_data"
data_basename = "unified_es_data_20251109"

# 前処理済みデータのファイルパス（20251120版）
preprocessed_files = {
    'es_data': os.path.join(preprocessed_dir, 'unified_es_data_20251120_es_data.pkl'),
    'tfidf_matrix': os.path.join(preprocessed_dir, 'unified_es_data_20251120_tfidf_matrix.npz'),
    'vectorizer': os.path.join(preprocessed_dir, 'unified_es_data_20251120_vectorizer.pkl'),
    'embeddings': os.path.join(preprocessed_dir, 'unified_es_data_20251120_embeddings.npy')
}

# 前処理済みデータが存在するかチェック
all_files_exist = all(os.path.exists(f) for f in preprocessed_files.values())

if all_files_exist:
    print("✅ 前処理済みデータを発見しました！")
    print(f"📂 保存先: {preprocessed_dir}")
    print("\n📁 利用可能なファイル:")
    for name, path in preprocessed_files.items():
        size_mb = os.path.getsize(path) / (1024 * 1024)
        print(f"  - {name}: {size_mb:.2f} MB")
    print("\n💡 これらのファイルを使用すると起動が高速化されます（数秒で完了）")
    USE_PREPROCESSED = True
else:
    print("ℹ️ 前処理済みデータが見つかりません")
    print("⏳ 初回起動のため、データの前処理を実行します（3-5分程度）")
    print(f"💾 完了後、前処理済みデータを以下に保存します:")
    print(f"   {preprocessed_dir}")
    USE_PREPROCESSED = False

## セル5: データ読み込みとアプリケーション起動

In [ ]:
import threading
import time
import pickle
import numpy as np
from scipy import sparse
import sys
import os

# srcディレクトリをPythonパスに追加（重要！）
sys.path.insert(0, os.path.join(os.getcwd(), 'src'))

# アプリケーションをインポート（SimCSE-JA版）
from app_simcse_ja import app, load_csv_data, es_data, vectorizer, tfidf_matrix, sentence_model

# ============================================
# ngrokの認証トークンを設定
# ============================================
# https://dashboard.ngrok.com/ でトークンを取得してください

NGROK_TOKEN = "YOUR_NGROK_TOKEN"  # ここにあなたのngrokトークンを入力

# ngrokインポートとセットアップ（リトライ付き）
print("🔧 ngrokをセットアップ中...")
for attempt in range(3):
    try:
        from pyngrok import ngrok
        ngrok.set_auth_token(NGROK_TOKEN)
        print("✅ ngrok認証トークンを設定しました")
        break
    except Exception as e:
        if attempt < 2:
            print(f"⚠️ リトライ {attempt + 1}/3...")
            time.sleep(5)
        else:
            print(f"❌ ngrokのセットアップに失敗しました: {e}")
            print("\n💡 解決方法:")
            print("1. ランタイムを再起動してもう一度試す")
            print("2. セル1を再実行してパッケージを再インストール")
            raise

# ============================================
# データを読み込み（前処理済みデータまたは新規）
# ============================================

# USE_PREPROCESSEDが定義されていない場合のデフォルト値を設定
if 'USE_PREPROCESSED' not in globals():
    USE_PREPROCESSED = False
    preprocessed_files = {}
    preprocessed_dir = ""
    print("ℹ️ セル4がスキップされたため、前処理済みデータを使用しません")
    print("⏳ 新規データの読み込みを実行します（3-5分程度）\n")

if USE_PREPROCESSED:
    print("\n⚡ 前処理済みデータを読み込み中...")
    print("💡 これにより起動が大幅に高速化されます")
    
    # グローバル変数にアクセスするためにappモジュールから取得
    import app_simcse_ja as app_module
    
    # es_dataを読み込み
    with open(preprocessed_files['es_data'], 'rb') as f:
        app_module.es_data = pickle.load(f)
    print(f"  ✅ es_data: {len(app_module.es_data)}件")
    
    # TF-IDF行列を読み込み
    app_module.tfidf_matrix = sparse.load_npz(preprocessed_files['tfidf_matrix'])
    print(f"  ✅ tfidf_matrix: {app_module.tfidf_matrix.shape}")
    
    # Vectorizerを読み込み
    with open(preprocessed_files['vectorizer'], 'rb') as f:
        app_module.vectorizer = pickle.load(f)
    print(f"  ✅ vectorizer")
    
    # セマンティックエンベディングを読み込み
    if os.path.exists(preprocessed_files['embeddings']):
        # SimCSE-JAモデルをロード（エンベディング計算には不要だが、モデルは必要）
        try:
            from sentence_transformers import SentenceTransformer
            if app_module.sentence_model is None:
                print("  📥 SimCSE-JAモデルをロード中...")
                app_module.sentence_model = SentenceTransformer('pkshatech/simcse-ja-bert-base-clcmlp')
                print("  ✅ モデルロード完了")
        except ImportError:
            print("  ⚠️ sentence-transformersが利用できません。セマンティック検索は無効です。")
        
        # エンベディングを読み込み
        app_module.es_data['semantic_embedding'] = list(np.load(preprocessed_files['embeddings']))
        print(f"  ✅ semantic_embeddings: {len(app_module.es_data['semantic_embedding'])}件")
    
    print("\n✅ 前処理済みデータの読み込みが完了しました（数秒で完了）")
    
else:
    print("\n📂 CSVデータを読み込み中...")
    print("⏳ 初回実行時はSimCSE-JAモデルのダウンロードに数分かかります...")
    print("⚡ セマンティックエンベディング生成は3-5分程度で完了します（バッチ処理最適化済み）")
    print("")
    load_csv_data(csv_path)
    print("\n✅ データの読み込みが完了しました")

# ============================================
# FastAPIアプリをバックグラウンドで起動
# ============================================
import uvicorn
from threading import Thread

def run_fastapi():
    uvicorn.run(app, host="0.0.0.0", port=8000, log_level="info")

fastapi_thread = Thread(target=run_fastapi, daemon=True)
fastapi_thread.start()

print("\n🚀 FastAPIアプリを起動しました (ポート8000)")

# ============================================
# ngrokトンネルを作成
# ============================================
time.sleep(3)  # FastAPIの起動を待つ

print("\n🌐 ngrokトンネルを作成中...")
try:
    public_url = ngrok.connect(8000)
    
    print("\n" + "="*60)
    print("🎉 アプリケーションが起動しました！（SimCSE-JA版）")
    print("="*60)
    print(f"\n🌐 公開URL: {public_url}")
    print("\n💡 上記のURLをクリックしてアプリにアクセスしてください")
    print("\n⚠️ このノートブックを閉じるとアプリも停止します")
    print("\n🔍 搭載機能:")
    print("  - TF-IDFキーワードマッチング")
    print("  - セマンティック検索（SimCSE-JA）で意味的類似性を理解")
    print("  - STAR構造分析でES品質を評価")
    
    if USE_PREPROCESSED:
        print("\n⚡ 前処理済みデータ使用:")
        print("  - 起動時間: 数秒（通常3-5分のところ）")
        print(f"  - データ保存先: {preprocessed_dir}")
    else:
        print("\n⚡ パフォーマンス:")
        print("  - バッチ処理最適化: エンベディング生成が40分→3-5分に短縮")
        print("  - SimCSE-JAモデル採用: 768次元の日本語特化エンベディング")
        print("  - GPU自動検出: T4有効時は30秒〜1分でエンベディング生成")
        print(f"\n💾 次回起動用に前処理済みデータを保存するには、セル6を実行してください")
    
    print("="*60)
except Exception as e:
    print(f"\n❌ ngrokトンネルの作成に失敗しました: {e}")
    print("\n💡 トラブルシューティング:")
    print("1. NGROK_TOKENが正しく設定されているか確認")
    print("2. https://dashboard.ngrok.com/ でトークンを再確認")
    print("3. ランタイムを再起動して最初からやり直す")
    raise

## セル6: 前処理済みデータを保存（オプション）

**次回起動を高速化するため、前処理済みデータをGoogle Driveに保存します。**

このセルを実行すると：
- es_data（前処理済みDataFrame）
- TF-IDF行列とVectorizer
- セマンティックエンベディング

が指定されたディレクトリに保存され、次回起動時に自動的に読み込まれます。

⚠️ **注意**: セル5でアプリケーションを起動した後に実行してください。

In [ ]:
import os
import pickle
import numpy as np
from scipy import sparse

# app_simcse_jaモジュールから前処理済みデータを取得
import app_simcse_ja as app_module

# 前処理済みデータが存在するか確認
if app_module.es_data is None or len(app_module.es_data) == 0:
    print("❌ エラー: データが読み込まれていません")
    print("💡 先にセル5を実行してアプリケーションを起動してください")
else:
    print("💾 前処理済みデータを保存中...")
    print(f"📂 保存先: {preprocessed_dir}")
    
    # 保存先ディレクトリを作成
    os.makedirs(preprocessed_dir, exist_ok=True)
    
    # 1. es_dataを保存
    print("\n  💾 es_data を保存中...")
    with open(preprocessed_files['es_data'], 'wb') as f:
        pickle.dump(app_module.es_data, f)
    size_mb = os.path.getsize(preprocessed_files['es_data']) / (1024 * 1024)
    print(f"  ✅ es_data 保存完了: {size_mb:.2f} MB")
    
    # 2. TF-IDF行列を保存
    if app_module.tfidf_matrix is not None:
        print("\n  💾 tfidf_matrix を保存中...")
        sparse.save_npz(preprocessed_files['tfidf_matrix'], app_module.tfidf_matrix)
        size_mb = os.path.getsize(preprocessed_files['tfidf_matrix']) / (1024 * 1024)
        print(f"  ✅ tfidf_matrix 保存完了: {size_mb:.2f} MB")
    
    # 3. Vectorizerを保存
    if app_module.vectorizer is not None:
        print("\n  💾 vectorizer を保存中...")
        with open(preprocessed_files['vectorizer'], 'wb') as f:
            pickle.dump(app_module.vectorizer, f)
        size_mb = os.path.getsize(preprocessed_files['vectorizer']) / (1024 * 1024)
        print(f"  ✅ vectorizer 保存完了: {size_mb:.2f} MB")
    
    # 4. セマンティックエンベディングを保存
    if 'semantic_embedding' in app_module.es_data.columns:
        print("\n  💾 semantic_embeddings を保存中...")
        embeddings_array = np.array(app_module.es_data['semantic_embedding'].tolist())
        np.save(preprocessed_files['embeddings'], embeddings_array)
        size_mb = os.path.getsize(preprocessed_files['embeddings']) / (1024 * 1024)
        print(f"  ✅ semantic_embeddings 保存完了: {size_mb:.2f} MB")
    else:
        print("\n  ⚠️ semantic_embeddingが見つかりません（スキップ）")
    
    # 保存完了サマリー
    print("\n" + "="*60)
    print("✅ 前処理済みデータの保存が完了しました！")
    print("="*60)
    print(f"\n📂 保存先: {preprocessed_dir}")
    print("\n📁 保存されたファイル:")
    total_size = 0
    for name, path in preprocessed_files.items():
        if os.path.exists(path):
            size_mb = os.path.getsize(path) / (1024 * 1024)
            total_size += size_mb
            print(f"  ✅ {name}: {size_mb:.2f} MB")
    print(f"\n💾 合計サイズ: {total_size:.2f} MB")
    print("\n💡 次回起動時にこれらのファイルが自動的に読み込まれます")
    print("⚡ 起動時間が 3-5分 → 数秒 に短縮されます！")
    print("="*60)